# Extraindo validade dos pdfs de Gêneros Alimentícios

### Esse notebook tem o objetivo de:
    - [X] Extrair a validade indicada nos pdfs
    - [X] Padronizar o formato para ISO8601 de intervalo temporal
    - [X] Criar uma função :: Path -> str que automatiza esse processo

### Prerequisitos:
    - [X] Baixar pelo menos um PDF para referencia

In [5]:
PDF = '../data/pdfs/generos-alimenticios/ARQUIVOPARAPORTAL.pdf'

In [6]:
import sys
sys.path.insert(0, '..')
from IPython.display import HTML

In [7]:
HTML(f'''
<iframe
    src="{PDF}#page=0"
    width=100%
    height=400
    >
</iframe>
''')

In [9]:
import re
from datetime import datetime
from PyPDF2 import PdfFileReader

## Extrai o texto da página 0

In [22]:
text_p0 = PdfFileReader(PDF).getPage(0).extractText()
text_p0.split('\n')[:10]

['CONTROLADORIA GERAL DO MUNICÍPIO',
 'http://www7.rio.rj.gov.br/cgm/tabelas/',
 'VALIDADE:    01/03/2017    a    15/03/2017',
 'ORD',
 'CÓDIGO',
 'GÊNEROS ALIMENTÍCIOS',
 'UNIDADE',
 'VALOR - R$',
 '1',
 '8905.01.001-40']

## Isola a parte que tem a validade

In [30]:
validade = re.findall(r'validade.*?(\d.+\d)',
                      text_p0,
                      flags=re.I
                      )[0]
validade

'01/03/2017    a    15/03/2017'

In [31]:
start, end = validade.split(' a ')
start, end

('01/03/2017   ', '   15/03/2017')

In [32]:
start, end = start.replace(' ', ''), end.replace(' ', '')
start, end

('01/03/2017', '15/03/2017')

## Normaliza o formato

In [33]:
start = (datetime
         .strptime(start, '%d/%m/%Y')
         .strftime('%Y-%m-%d'))
end   = (datetime
         .strptime(end, '%d/%m/%Y')
         .strftime('%Y-%m-%d'))

start, end

('2017-03-01', '2017-03-15')

In [34]:
timespan = '/'.join((start, end))
timespan

'2017-03-01/2017-03-15'

## Fatorar numa função

In [35]:
# Returns ISO8601 time interval
def extract_validity(path : str) -> str:
    # open and extract first page text
    text_p0 = PdfFileReader(path).getPage(0).extractText()

    # returns like: '16/09/2017    a    30/09/2017'
    validade = re.findall(r'validade.*?(\d.+\d)', text_p0, flags=re.I)[0]

    # ['16/09/2017', 'a', '30/09/2017']
    start, end = validade.split(' a ')
    start = start.replace(' ', '')
    end   = end.replace(' ', '')

    # convert to ISO8601 date format
    start = datetime.strptime(start, '%d/%m/%Y').strftime('%Y-%m-%d')
    end   = datetime.strptime(end, '%d/%m/%Y').strftime('%Y-%m-%d')

    # ISO ISO8601 time interval format
    return '/'.join((start, end))

In [36]:
extract_validity(PDF)

'2017-03-01/2017-03-15'